# Task 3

In [1]:
# !pip install transformers -q
# !pip install datasets -q


In [2]:
# !pip install numpy -q
# !pip install matplotlib -q

('11.3', '1.12.0+cu113')

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import csv
from tqdm.notebook import tqdm_notebook as pbar
content = []
with open('QA dataset/Question-answering.csv','r',encoding="iso8859_16") as f:
# with open('../input/qadataset/Question-answering.csv','r',encoding="iso8859_16") as f:
# with open('/content/drive/My Drive/Colab Notebooks/QA dataset/Question-answering.csv','r',encoding="iso8859_16") as f:
    reader = csv.reader(f)
    header = True
    c = 0
    for line in reader:   
        if header:
            header = False
            continue
        content.append(line)
print(len(content))

9749


In [5]:
import datasets
import pandas as pd
import shlex

In [6]:

def convert_to_dataset(content, start, end):
    '''
    construct context-question pairs and corresponding answers as a huggingface dataset
    '''
    data_rows = []
    index_no = start
    for line in pbar(content[start:end]):
        context = str(line[0])
        qasets = line[1]
        qasets = qasets.replace('{<|','')
        qasets = qasets.replace('|>}','')
        qasetlist = qasets.split('|>, <|')
        for qaset in qasetlist:
            question_wrapper, rest = tuple(qaset.split(', "Answers"', maxsplit=1))
            _, question = tuple(question_wrapper.split(' -> '))
            if(question[0] != '"' or question[-1] != '"'):
                print("Error: ", question)
                print((question[1]), (question[-1]))
                continue
            answers_wrapper, rest = tuple(rest.split('}, ', maxsplit=1))
            _, answers = tuple(answers_wrapper.split(' -> {'))
            answers = shlex.split(answers)
            answers_start_wrapper, rest = tuple(rest.split('},', maxsplit=1))
            _, answers_start = tuple(answers_start_wrapper.split(' -> '))
            answers_start = [int(x) for x in answers_start.replace('{','').replace('}','').split(',')]
            question_id_wrapper = rest
            _, question_id = tuple(question_id_wrapper.split(' -> '))

            answer_dict = {'text':[], 'answer_start':[]}
            for i in range(len(answers)):
                try:
                    answer_dict['text'].append(answers[i])
                    answer_dict['answer_start'].append(answers_start[i])
                except:
                    print(index_no, answers)
                    continue
            data_rows.append({
                'context': context,
                'question': question[1:-1],
                # 'answer_text': answers[i][1:-1],
                # 'answer_start': answers_start[i],
                # 'answer_end': answers_start[i] + len(answers[i][1:-1]),
                'answers': answer_dict,
                'id': question_id[1:-1]
            })
        # break
        index_no += 1

    return pd.DataFrame(data_rows)


''' Create pandas dataframes for train, test and validation '''

In [7]:
train_df = convert_to_dataset(content, 6001, 7001)
train_dataset = datasets.Dataset.from_dict(train_df)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [8]:
val_df = convert_to_dataset(content, 9400, 9600)
val_dataset = datasets.Dataset.from_dict(val_df)

  0%|          | 0/200 [00:00<?, ?it/s]

In [9]:
test_df = convert_to_dataset(content, 9600, 9749)
test_dataset = datasets.Dataset.from_dict(test_df)

  0%|          | 0/149 [00:00<?, ?it/s]

In [10]:
MODEL_NAME = 'bert-base-uncased'

In [11]:
raw_datasets = datasets.dataset_dict.DatasetDict({"train":train_dataset,"validation":val_dataset, "test":test_dataset})

In [12]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answers', 'id'],
        num_rows: 4781
    })
    validation: Dataset({
        features: ['context', 'question', 'answers', 'id'],
        num_rows: 1323
    })
    test: Dataset({
        features: ['context', 'question', 'answers', 'id'],
        num_rows: 642
    })
})

In [17]:
from transformers import AutoTokenizer

model_checkpoint = MODEL_NAME
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [18]:
tokenizer.is_fast

True

In [19]:
max_length = 384
stride = 128


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [20]:
train_dataset = raw_datasets["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)
len(raw_datasets["train"]), len(train_dataset)

  0%|          | 0/5 [00:00<?, ?ba/s]

(4781, 4862)

In [21]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [22]:
validation_dataset = raw_datasets["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)
len(raw_datasets["validation"]), len(validation_dataset)

  0%|          | 0/2 [00:00<?, ?ba/s]

(1323, 1323)

In [23]:
test_dataset = raw_datasets["test"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_datasets["test"].column_names,
)
len(raw_datasets["test"]), len(validation_dataset)

  0%|          | 0/1 [00:00<?, ?ba/s]

(642, 1323)

In [24]:
import torch
from transformers import AutoModelForQuestionAnswering

In [25]:
from datasets import load_metric
import collections

metric = load_metric("squad")

In [26]:
# str(type(metric))

In [27]:
# theoretical_answers = [
#     {"id": ex["id"], "answers": ex["answers"]} for ex in small_eval_set
# ]

In [28]:
from tqdm.auto import tqdm

n_best = 20
max_answer_length=30
def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None or offsets[start_index] is [] or offsets[end_index] is []:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue
                    try:
                        answer = {
                            "text": context[offsets[start_index][0] : offsets[end_index][1]],
                            "logit_score": start_logit[start_index] + end_logit[end_index],
                        }
                        answers.append(answer)
                    except:
                        print(start_index, end_index, len(offsets), offsets[start_index], offsets[end_index], len(context))
#                         answer = {
#                             "text": context[offsets[start_index][0] : offsets[end_index][1]],
#                             "logit_score": start_logit[start_index] + end_logit[end_index],
#                         }
#                         answers.append(answer)
                        pass

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    print('hey!')
    try:
      return metric.compute(predictions=predicted_answers, references=theoretical_answers)
    except:
      print('handling exception...')
      metric = load_metric('squad')
      return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [29]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [30]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

train_dataset.set_format("torch")
validation_set = validation_dataset.remove_columns(["example_id", "offset_mapping"])
validation_set.set_format("torch")

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    validation_set, collate_fn=default_data_collator, batch_size=8
)

In [31]:
train_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 4862
})

In [32]:
# model = AutoModelForQuestionAnswering.from_pretrained('bert-base-uncased')
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [33]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [34]:
from transformers import get_scheduler

num_train_epochs = 10
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
from tqdm.auto import tqdm
import torch
import numpy as np

    
logs = {}
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for step, batch in enumerate(train_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    start_logits = []
    end_logits = []
    # metric = load_metric('glue', 'mrpc')
    for batch in tqdm(eval_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        start_logits.append(outputs.start_logits.cpu().numpy())
        end_logits.append(outputs.end_logits.cpu().numpy())

    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    start_logits = start_logits[: len(validation_dataset)]
    end_logits = end_logits[: len(validation_dataset)]

    metrics = compute_metrics(
        start_logits, end_logits, validation_dataset, raw_datasets["validation"]
    )
    print(f"epoch {epoch}:", metrics)
    
    logs['epoch'] = metrics

    # Save and upload
    torch.save(model, 'model_{}.pt'.format(epoch))
    break



  0%|          | 0/6080 [00:00<?, ?it/s]

### Model in action

In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = torch.load('model_9.pt').to(device)
question_answerer = pipeline("question-answering", model=model_checkpoint, tokenizer=tokenizer, device=0)

context = test_df.iloc[0]['context']
question = test_df.iloc[0]['question']
ans = question_answerer(question=question, context=context)

In [ ]:
ans